# Plotting Maps &mdash; Casos COVID-19
## Requisitos
Necesitarás instalar el paquete `cartopy` Python así como algunas dependencias adicionales. Siga las instrucciones [aqui](https://scitools.org.uk/cartopy/docs/latest/installing.html#installing) para la instalación. Tenga en cuenta que, a menos que especifique explícitamente que no actualice los paquetes instalados actualmente, es posible que acabe con un entorno que ya no esté configurado para este libro; si esto ocurre, reconstruya su entorno como hizo en el capítulo 1. Asegúrate de revisar estas preguntas de StackOverflow para [pip](https://stackoverflow.com/questions/2861183/upgrade-package-without-upgrading-dependencies-using-pip).

## Data Wrangling
En este cuaderno, utilizaremos el Centro Europeo para la Prevención y el Control de las Enfermedades (ECDC) [número diario de nuevos casos notificados de COVID-19 por país conjunto de datos mundial](https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide) archivo más reciente. Si esto ya no funciona, lea los datos del capítulo que es una instantánea recogida el 19 de septiembre de 2020:

```python
df = pd.read_csv('data/covid19_cases.csv')
```

En primer lugar, recopilamos los datos y los resumimos para obtener los casos acumulados por país:

In [4]:
import pandas as pd

# léase en los últimos datos COVID-19 del Centro Europeo para la Prevención y el Control de las Enfermedades (ECDC)
df = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv').rename(columns={'cases_weekly': 'cases'})

# resumir los datos COVID-19 para obtener los casos acumulados por país
total_cases_per_country = df.groupby('countriesAndTerritories').cases.sum().rename(lambda x: x.replace('_', ' '), axis=0)

## Trazado de países con Matplotlib y Cartopy

In [5]:
%matplotlib inline
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import EngFormatter, MaxNLocator
import numpy as np
import color_utils

# build layout
fig = plt.figure(figsize=(20, 15))
gs = fig.add_gridspec(1, 10)
map_subplot = fig.add_subplot(gs[0, :9], projection=ccrs.PlateCarree())
cbar_subplot = fig.add_subplot(gs[0, 9])

# selecciona el mapa de color; recuerda que es un callable así que colormap(0.5)
# nos dará el color en el punto medio, por lo que tendremos que escalar
# los casos para que estén en [0, 1] (dividiremos por el número máximo de casos)
colormap = cm.get_cmap('Oranges')

# nombres que son diferentes en el shapefile frente a los datos COVID-19
alternate_names = {
    'Tanzania': 'United Republic of Tanzania',
    'The Bahamas': 'Bahamas',
    'Falkland Islands': 'Falkland Islands (Malvinas)',
    'Somaliland': 'Somalia',
    'Republic of Macedonia': 'North Macedonia',
    'East Timor': 'Timor Leste',
    'Ivory Coast': 'Cote dIvoire',
    'Guinea-Bissau': 'Guinea Bissau',
    'Republic of the Congo': 'Democratic Republic of the Congo',
    'eSwatini': 'Eswatini',
    'The Gambia': 'Gambia',
    "People's Republic of China": 'China',
    'Brunei': 'Brunei Darussalam',
    'Czech Republic': 'Czechia'
}

# obtener shapefile para dibujar el mapa
countries_shp = shpreader.natural_earth(
    resolution='110m', category='cultural', name='admin_0_countries'
)

# añada países al gráfico, sombreándolos según el número de casos de COVID-19
for country in shpreader.Reader(countries_shp).records():
    country_name = country.attributes['NAME_EN']
    country_name = alternate_names.get(country_name, country_name)
    map_subplot.add_geometries(
        [country.geometry], 
        ccrs.PlateCarree(),
        facecolor=colormap(total_cases_per_country.get(country_name, 0) / total_cases_per_country.max()),
        label=country_name
    )

# crear una barra de color para el mapa
cbar = color_utils.draw_cmap(
    colormap, 
    values=np.array([np.concatenate([[0], total_cases_per_country])]), 
    orientation='vertical', 
    ax=cbar_subplot
)
cbar.ax.yaxis.set_major_formatter(EngFormatter())
cbar.ax.yaxis.set_major_locator(MaxNLocator(5))
max_full_day = pd.to_datetime(df.dateRep, format='%d/%m/%Y').max()
cbar.ax.set_title(f'COVID-19 cases\n(as of {max_full_day.strftime("%b %d, %Y")})')

# eliminar el recuadro alrededor del mapa
for spine in map_subplot.spines:
    map_subplot.spines[spine].set_visible(False)

ModuleNotFoundError: No module named 'cartopy'